In [ ]:
import ipywidgets
import matplotlib.pyplot as plt 
import numpy as np
from numpy import absolute, mean, std
import pandas as pd
import seaborn as sns
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold

In [ ]:
df = pd.read_csv("nfl_grades.csv")
drop_cols = ['Unnamed: 0', 'game-id', 'home-spec', 'away-spec']
df.drop(columns=['Unnamed: 0', 'game-id'], inplace=True)
targets = ['home-score', 'away-score']
y = df[targets]
X = df.drop(columns=targets)


y = y.values
X = X.values

# get the model
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(26*26, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(100, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(50, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs))
    model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])
    return model
 
# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
    results = list()
    n_inputs, n_outputs = X.shape[1], y.shape[1]
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
        # prepare data
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        # define model
        model = get_model(n_inputs, n_outputs)
        # fit model
        model.fit(X_train, y_train, verbose=0, epochs=100)
        # evaluate model on test set
        #mae = model.evaluate(X_test, y_test, verbose=0)
        predictions = model.predict(X_test)
        mae = 0
        for i in range(len(predictions)):
            mae+= absolute(y_test[i,0]-predictions[i,0])+absolute(y_test[i,1]-predictions[i,1])
        
        mae/= len(predictions)
        # store result
        print('>%.3f' % mae)
        results.append(mae)
    return results
 

# evaluate model
results = evaluate_model(X, y)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(results), std(results)))